# 처리량이 높은 VPN 빌드

Google Cloud 및 기타 클라우드 또는 온프레미스 시스템 간의 보안 통신은 공통적인 필수 요구사항입니다. 다행히 Google Cloud에서는 이러한 목표를 달성하기 위한 안전한 인터넷 보안 프로토콜(IPsec) 가상 사설망(VPN)을 쉽게 만들 수 있습니다. 단일 터널이 필요한 처리량을 제공하지 않을 경우 Google Cloud는 여러 터널에 트래픽을 원활하게 분산하여 추가 대역폭을 제공할 수 있습니다.

### 목표
1. VPN 만들기
Google Cloud 네트워크를 시뮬레이션하는 cloud라는 이름의 가상 프라이빗 클라우드(VPC)와 외부 네트워크를 시뮬레이션하는 on-prem(온프레미스)이라는 이름의 VPC를 생성.</br>
cloud VPC에 사용할 VPN 게이트웨이, 전달 규칙, 주소를 생성.</br>
새 VPN을 위한 터널을 만들고 이를 통해 트래픽을 라우팅.</br>
on-prem VPC로 VPN 생성 프로세스를 반복하여 두 번째 VPN을 만듬.</br>
2. VPN 테스트하기
Compute Engine을 사용하여 처리량 부하 테스트를 위한 가상 머신(VM)을 만듭니다.
iperf를 사용하여 단일 VPN의 처리량 속도를 테스트합니다.

### 클라우드 VPC 만들기
이 섹션에서 수행할 작업은 다음과 같습니다.

클라우드 프로덕션 네트워크를 시뮬레이션하는 VPC를 만듭니다.
일반적인 유형의 트래픽이 VPC를 통과하도록 허용합니다.
호스트 배포를 위한 서브넷을 만듭니다.
Cloud Shell을 설정한 후 다음을 실행하여 Google Cloud 프로젝트와 연결된 cloud라는 이름의 커스텀 VPC를 만듭니다.
> gcloud compute networks create cloud --subnet-mode custom

이 VPC의 경우 기본값이 아닌 IP 주소를 사용할 수 있지만 기본 방화벽 규칙은 포함하지 않습니다.

부하 테스트 중에 VM과 통신하려면 시큐어 셸이 필요하므로 다음을 실행하여 SSH 및 icmp를 사용 설정합니다.
>gcloud compute firewall-rules create cloud-fw --network cloud --allow tcp:22,tcp:5001,udp:5001,icmp"

다음을 실행하여 이 VPC 내에 서브넷을 만들고 리전과 IP 범위를 지정합니다.
>gcloud compute networks subnets create cloud-east --network cloud \
    --range 10.0.1.0/24 --region REGION2

### 온프레미스 VPC 만들기

1. Cloud Shell에서 다음을 실행하여 프로젝트와 연결된 on-prem이라는 이름의 새 커스텀 서브넷 VPC를 만듭니다.
>gcloud compute networks create on-prem --subnet-mode custom

2. 부하 테스트 중에 VM과 통신하려면 시큐어 셸이 필요하므로 다음을 실행하여 on-prem VPC의 호스트에 SSH 및 icmp를 사용 설정합니다.
>gcloud compute firewall-rules create on-prem-fw --network on-prem --allow tcp:22,tcp:5001,udp:5001,icmp

3. 다음 명령어를 사용해 리전의 서브넷 프리픽스를 지정합니다.
>gcloud compute networks subnets create on-prem-central \
    --network on-prem --range 192.168.1.0/24 --region REGION


### VPN 게이트웨이 만들기
안전한 외부 통신을 위해 각 환경에는 VPN 게이트웨이가 필요합니다. 다음 단계에 따라 클라우드 및 on-prem VPC의 초기 게이트웨이를 만듭니다.

1. Cloud Shell에서 on-prem VPC 및 us-central1 리전에 on-prem-gw1이라는 이름의 VPN 게이트웨이를 만듭니다.
>gcloud compute target-vpn-gateways create on-prem-gw1 --network on-prem --region us-central1

2. 이제 cloud VPC 및 europe-west1 리전에 cloud-gw1이라는 이름의 VPN을 만듭니다.
>gcloud compute target-vpn-gateways create cloud-gw1 --network cloud --region europe-west1


### 로컬 네트워크와 Google Cloud 네트워크 사이에 경로 기반 VPN 터널 만들기

각 VPN 게이트웨이에 정적 외부 IP 주소가 있어야 VPC 외부 시스템과 통신할 수 있습니다. 이제 클라우드 및 on-prem VPC의 IP 주소와 경로를 만듭니다.

1. Cloud Shell에서 cloud-gw1 VPN 게이트웨이의 IP를 할당합니다.
>gcloud compute addresses create cloud-gw1 --region europe-west1

2. 그런 다음 on-prem-gw1 VPN 게이트웨이의 IP를 할당합니다.
>

3. 이후 명령어에서 다시 찾을 필요가 없도록 게이트웨이 주소를 저장합니다.
먼저 cloud-gw1 게이트웨이에서 다음 명령어를 실행합니다.
>cloud_gw1_ip=$(gcloud compute addresses describe cloud-gw1 \
    --region europe-west1 --format='value(address)')

그런 다음 on-prem-gw1 게이트웨이에서 다음 명령어를 실행합니다.
>on_prem_gw_ip=$(gcloud compute addresses describe on-prem-gw1 \
    --region us-central1 --format='value(address)')

4. 이제 cloud VPC의 IPsec에 대한 전달 규칙을 만듭니다. 양방향으로 전달 규칙을 만들어야 합니다.
cloud-gw1에서 보안 페이로드 캡슐화(ESP) 프로토콜을 전달합니다.
>gcloud compute forwarding-rules create cloud-1-fr-esp --ip-protocol ESP \
    --address $cloud_gw1_ip --target-vpn-gateway cloud-gw1 --region europe-west1

cloud-gw1에서 UDP:500 트래픽을 전달합니다.
>gcloud compute forwarding-rules create cloud-1-fr-udp500 --ip-protocol UDP \
    --ports 500 --address $cloud_gw1_ip --target-vpn-gateway cloud-gw1 --region europe-west1

cloud-gw1에서 UDP:4500 트래픽을 전달합니다.
>gcloud compute forwarding-rules create cloud-fr-1-udp4500 --ip-protocol UDP \
    --ports 4500 --address $cloud_gw1_ip --target-vpn-gateway cloud-gw1 --region europe-west1

5. 동일한 메서드를 사용해 on-prem VPC의 IPsec 터널에 대한 방화벽 전달 규칙을 만듭니다. 이 단계를 통해 IPsec 터널이 방화벽을 종료할 수 있습니다.

on-prem-gw1에서 ESP 프로토콜을 전달합니다.
>gcloud compute forwarding-rules create on-prem-fr-esp --ip-protocol ESP \
    --address $on_prem_gw_ip --target-vpn-gateway on-prem-gw1 --region us-central1

on-prem-gw1에서 IPsec 터널을 설정하는 데 사용된 UDP:500 트래픽을 전달합니다.
>gcloud compute forwarding-rules create on-prem-fr-udp500 --ip-protocol UDP --ports 500 \
    --address $on_prem_gw_ip --target-vpn-gateway on-prem-gw1 --region us-central1

on-prem-gw1에서 암호화된 트래픽을 전송하는 UDP:4500 트래픽을 전달합니다.
>gcloud compute forwarding-rules create on-prem-fr-udp4500 --ip-protocol UDP --ports 4500 \
    --address $on_prem_gw_ip --target-vpn-gateway on-prem-gw1 --region us-central1

보통은 on-prem-tunnel1 및 cloud-tunnel1 터널을 만들고 검증하는 다음 단계에서 보안 비밀을 생성합니다. 보안 비밀을 만들어 안전하게 저장하는 방법에 관한 자세한 내용은 Secret Manager 개념 개요 가이드를 확인하세요. 지금은 'sharedsecret' 문자열을 사용합니다.

on-prem-tunnel1 로컬 네트워크와 cloud-tunnel1 클라우드 기반 네트워크의 터널을 만듭니다. 각 네트워크에는 VPN 게이트웨이가 있어야 하고 보안 비밀이 일치해야 합니다. 다음 두 명령어에서 [MY_SECRET]을 'sharedsecret'으로 바꿉니다. 프로덕션 시나리오에서는 자신이 생성한 보안 비밀로 바꿉니다.

6. on-prem에서 cloud로 통하는 VPN 터널을 만듭니다.
>gcloud compute vpn-tunnels create on-prem-tunnel1 --peer-address $cloud_gw1_ip \
    --target-vpn-gateway on-prem-gw1 --ike-version 2 --local-traffic-selector 0.0.0.0/0 \
    --remote-traffic-selector 0.0.0.0/0 --shared-secret=[MY_SECRET] --region us-central1

7. 클라우드에서 온프렘으로 통하는 VPN 터널을 만듭니다.
>gcloud compute vpn-tunnels create cloud-tunnel1 --peer-address $on_prem_gw_ip \
    --target-vpn-gateway cloud-gw1 --ike-version 2 --local-traffic-selector 0.0.0.0/0 \
    --remote-traffic-selector 0.0.0.0/0 --shared-secret=[MY_SECRET] --region europe-west1

게이트웨이를 만들고 터널을 빌드했으므로 이제 서브넷에서 두 개의 터널을 통과하는 경로를 추가해야 합니다.

8. on-prem VPC에서 cloud 10.0.1.0/24 범위까지의 트래픽을 터널로 라우팅합니다.
>gcloud compute routes create on-prem-route1 --destination-range 10.0.1.0/24 \
    --network on-prem --next-hop-vpn-tunnel on-prem-tunnel1 \
    --next-hop-vpn-tunnel-region us-central1

9. cloud VPC에서 on-prem 192.168.1.0/24 범위까지의 트래픽을 터널로 라우팅합니다.
>gcloud compute routes create cloud-route1 --destination-range 192.168.1.0/24 \
    --network cloud --next-hop-vpn-tunnel cloud-tunnel1 --next-hop-vpn-tunnel-region europe-west1